In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, set_seed
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 31.3 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=988532a42e2bce68e39ae7e65ff14cb3aa8886db457b08f3a1afc5f350eef9f9
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8fc84ea6860a4462a724a5d7c9ab5b70b612bcd9bfd5282681ee96b1caf0ffa8
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [4]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


In [5]:
df_org = pd.read_csv("/content/drive/MyDrive/CSE6740/dropna_processed.csv")

In [6]:
summary_and_transcript = df_org[['transcript', 'summary']]
print(len(summary_and_transcript))
summary_and_transcript.dropna()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# print(summary_and_transcript.iloc[4, 1])
# print(len(summary_and_transcript.iloc[4, 1]))
drops = []
for i in range(len(summary_and_transcript)):
    if isinstance(summary_and_transcript.iloc[i,0],str):
        continue
    else:
        drops.append(i)
summary_and_transcript.drop(drops, inplace=True)
print(len(summary_and_transcript))

5631
4983


<ipython-input-6-80bd03659bfc>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_and_transcript.drop(drops, inplace=True)


In [7]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
def abs_summ(text, num_sentences=15):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

# Example usage
# original_text = summary_and_transcript.iloc[9,0]
# print(original_text)
# summary = abs_summ(original_text)
# print("Summary:", summary)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

cuda
Tue Apr 16 19:31:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              23W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------------------

In [ ]:
bart_summaries_1 = pd.DataFrame(columns=['Abstractive', 'Lex-Bart', 'Original'])

# Generate summaries for each chunk
def final_summ(dataframe):
    max_length = 1024
    # Load pre-trained model and tokenizer on the specified device
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)
    for i in range(len(dataframe)):
        if i ==1:
            print(bart_summaries_1)
        if i % 100 == 0:
            print(i)
        if i == 500:
            return bart_summaries_1
        transcript = summary_and_transcript.iloc[i,0]
        original = summary_and_transcript.iloc[i,1]
        abstractive_summary = abs_summ(transcript)
        ted_talk = [abstractive_summary[i: i+max_length] for i in range(0, len(abstractive_summary), max_length)]
        summaries = []
        for sentence in ted_talk:
            tokenized_chunk = tokenizer.encode(sentence, return_tensors='pt', max_length=max_length, truncation=True).to(device)
            # summary_ids = model.generate(tokenized_chunk)
            summ = model.generate(tokenized_chunk,num_beams=4, min_length=30, max_length= 200, early_stopping=True)
            summary = tokenizer.decode(summ[0], skip_special_tokens=True)
            summaries.append(summary)
        lex_bart = " ".join(summaries)
        bart_summaries_1.loc[i] = [abstractive_summary, lex_bart, original]
    return bart_summaries_1

bart_summaries_1 = final_summ(summary_and_transcript)
print(len(bart_summaries_1)
bart_summaries_1.to_csv("/content/drive/MyDrive/CSE6740/bart_summaries.csv")

In [11]:
print(bart_summaries_1.iloc[0,0])
print(bart_summaries_1.iloc[0,1])
print(bart_summaries_1.iloc[0,2])

I started thinking about my father and my grandfather and my great-grandfather, and I realized that I had all of these Teds going through my bloodstream -- (Laughter) that I had to consider this "my element." This is what you see. And when I started my blog, it was really this one goal -- I said, "I am not going to be famous to the world, but I could be famous to people on the Internet." And I won it -- I reached all of these people, and I had tens of thousands of people reading about my life every day. Because the joke in this is that this person is not a tyrant, this person is so loving and so sweet that he lets me dress him up and post pictures of him to my blog. These things resonate with us, and, if you think about blogs, you think of high art blogs, the history paintings about, you know, all the biblical stories, and then you have this. And we started with a slide of my Teds, and I had to add this slide, because I knew the minute I showed this, my mom -- my mom will see this, bec

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

t5_summaries = pd.DataFrame(columns=['Abstractive', 'Lex-t5', 'Original'])

def summarize_dataframe_with_t5(dataframe, num_sentences=15):
    tokenizer = T5Tokenizer.from_pretrained('t5-base')
    model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
    for i in range(len(dataframe)):
        if i % 100 == 0:
            print(i)
        if i == 500:
            return t5_summaries
        transcript = summary_and_transcript.iloc[i,0]
        original = summary_and_transcript.iloc[i,1]
        abstractive_summary = abs_summ(transcript, 10)
        input_ids = tokenizer.encode("summarize: " + abstractive_summary, return_tensors="pt", max_length=512, truncation=True).to(device)
        summary_ids = model.generate(input_ids, num_beams=4, max_length=200, early_stopping=True)
        lex_t5 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        t5_summaries.loc[i] = [abstractive_summary, lex_t5, original]
    return t5_summaries

t5_summaries = summarize_dataframe_with_t5(summary_and_transcript)

In [14]:
print(len(t5_summaries))
t5_summaries.to_csv("/content/drive/MyDrive/CSE6740/t5_summaries.csv")

500


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_perplexity(model, tokenizer, text):
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(input_ids)
    loss = outputs.loss
    return torch.exp(loss).item()

def calculate_cosine_similarity(vector1, vector2):
    # Reshape vectors for compatibility with cosine_similarity function
    vector1 = np.array(vector1).reshape(1, -1)
    vector2 = np.array(vector2).reshape(1, -1)
    # Calculate cosine similarity
    similarity = cosine_similarity(vector1, vector2)
    return similarity[0][0]

def compute_overall_score(df, model, tokenizer):
    perplexity_scores = []
    cosine_similarity_scores = []

    for index, row in df.iterrows():
        abstractive_summary = row['Abstractive']
        original_text = row['Original']
        lex_t5_summary = row['Lex-t5']
        # perplexity = calculate_perplexity(model, tokenizer, lex_t5_summary)
        # perplexity_scores.append(perplexity)
        original_text_embedding = model.get_input_embeddings()(torch.tensor(tokenizer.encode(original_text, return_tensors='pt')).to(device)).mean(dim=1).squeeze().detach().cpu().numpy()
        lex_t5_summary_embedding = model.get_input_embeddings()(torch.tensor(tokenizer.encode(lex_t5_summary, return_tensors='pt')).to(device)).mean(dim=1).squeeze().detach().cpu().numpy()
        cosine_sim = calculate_cosine_similarity(original_text_embedding, lex_t5_summary_embedding)
        cosine_similarity_scores.append(cosine_sim)

    # average_perplexity = np.mean(perplexity_scores)
    average_cosine_similarity = np.mean(cosine_similarity_scores)

    return cosine_similarity_scores, average_cosine_similarity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
overall_t5, avg_t5 = compute_overall_score(t5_summaries, model, tokenizer)
print("T5 Score:", overall_score)

In [28]:
print(overall_t5)
print(avg_t5)

[0.7363961, 0.6694827, 0.53489107, 0.50890034, 0.6844784, 0.50570303, 0.59034884, 0.583771, 0.6123754, 0.33063924, 0.67821485, 0.48929858, 0.54496175, 0.6723555, 0.634275, 0.6079096, 0.61761594, 0.4713366, 0.5916115, 0.5218723, 0.5596863, 0.6551231, 0.6471044, 0.45891553, 0.65796185, 0.6483439, 0.64413345, 0.6677161, 0.44425392, 0.6116297, 0.6188425, 0.5416427, 0.45553893, 0.65081084, 0.5765766, 0.4535401, 0.6917925, 0.58510804, 0.5137311, 0.66216946, 0.5966382, 0.5909243, 0.5504982, 0.52516854, 0.5745502, 0.61877716, 0.5828244, 0.66053313, 0.70621085, 0.5933168, 0.60188425, 0.56636024, 0.47541913, 0.7117386, 0.55044866, 0.51752424, 0.6317075, 0.6642766, 0.46108997, 0.6279907, 0.58274716, 0.6758913, 0.6840514, 0.7334981, 0.69077164, 0.7163217, 0.5470681, 0.54765, 0.59228146, 0.46396518, 0.7119931, 0.70878834, 0.69455016, 0.5764084, 0.5992675, 0.65145266, 0.66257524, 0.6205771, 0.37335354, 0.6018231, 0.43644804, 0.6385575, 0.6767786, 0.6006348, 0.60389566, 0.59939384, 0.5211131, 0.55670

In [ ]:
def compute_overall_score(df, model, tokenizer):
    perplexity_scores = []
    cosine_similarity_scores = []

    for index, row in df.iterrows():
        abstractive_summary = row['Abstractive']
        original_text = row['Original']
        bart_summary = row['Lex-Bart']
        # perplexity = calculate_perplexity(model, tokenizer, bart_summary)
        # perplexity_scores.append(perplexity)
        original_text_embedding = model.get_input_embeddings()(torch.tensor(tokenizer.encode(original_text, return_tensors='pt')).to(device)).mean(dim=1).squeeze().detach().cpu().numpy()
        bart_summary_embedding = model.get_input_embeddings()(torch.tensor(tokenizer.encode(bart_summary, return_tensors='pt')).to(device)).mean(dim=1).squeeze().detach().cpu().numpy()
        cosine_sim = calculate_cosine_similarity(original_text_embedding, bart_summary_embedding)
        cosine_similarity_scores.append(cosine_sim)
    # average_perplexity = np.mean(perplexity_scores)
    average_cosine_similarity = np.mean(cosine_similarity_scores)
    return cosine_similarity_scores, average_cosine_similarity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

# Compute the overall score
overall_score, avg = compute_overall_score(bart_summaries_1, model, tokenizer)

In [26]:
print(overall_score)
print(avg)

[0.8657371, 0.8442575, 0.76928145, 0.868539, 0.71671665, 0.7258701, 0.7977317, 0.8380244, 0.791854, 0.6318065, 0.8035615, 0.674251, 0.77203256, 0.8426492, 0.7770408, 0.7728736, 0.726523, 0.75452554, 0.69017494, 0.73740023, 0.7717066, 0.81605273, 0.8176382, 0.72610176, 0.8413888, 0.82535267, 0.8431218, 0.8380389, 0.7422627, 0.7557462, 0.7730943, 0.82554114, 0.67042166, 0.74485075, 0.7788523, 0.72860885, 0.8296911, 0.74877137, 0.7256991, 0.87530327, 0.7284474, 0.8102125, 0.7293006, 0.71124536, 0.6521554, 0.79009295, 0.8123746, 0.7740084, 0.8664996, 0.84985214, 0.73205435, 0.78139573, 0.6875516, 0.87162447, 0.68131495, 0.703416, 0.8598942, 0.8377553, 0.66375816, 0.7917751, 0.8176939, 0.77404416, 0.84254277, 0.87158346, 0.8014062, 0.84652746, 0.7746314, 0.7349194, 0.79600865, 0.6892609, 0.8149552, 0.8466179, 0.87138426, 0.6847086, 0.8136245, 0.8065281, 0.7925178, 0.73966914, 0.5784042, 0.77047527, 0.7429849, 0.8034038, 0.799767, 0.6331129, 0.74131066, 0.8210633, 0.7127347, 0.8299973, 0.810

In [ ]:
%%python -m pip install openai==1.6.1


In [59]:
import openai
from openai import OpenAI
import os

client = OpenAI()

gpt_summaries = pd.DataFrame(columns=['Summary', 'Original'])

for i in range(len(summary_and_transcript)):
    if i % 100 == 0:
        print(i)
    if i == 500:
        break
    transcript = summary_and_transcript.iloc[i,0]
    original = summary_and_transcript.iloc[i,1]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "system", "content": "Your goal is to summarize the transcript within 4 sentences."},
         {"role": "user", "content": transcript}],
          temperature=0.7, max_tokens=150, top_p=1)
    gpt_summaries.loc[i] = [response.choices[0].message.content, original]

2
2
2
2
2


KeyboardInterrupt: 

In [60]:
print(len(gpt_summaries))
print(gpt_summaries.iloc[0,0])
print(gpt_summaries.iloc[0,1])

496
Mena Trott, a blogger, shares her journey of starting a blog in 2001 and the personal stories she shared, including a touching story of a fellow blogger who documented her battle with cancer. She discusses the power of blogging in connecting people and sharing personal experiences, emphasizing the importance of leaving a digital legacy through blogs. Mena reflects on the impact of blogs in shaping personal narratives and creating connections across the world, highlighting the significance of sharing stories through this medium. She encourages the audience to consider the power of blogs and to embrace the platform as a means of personal expression and connection.
The founding mother of the blog revolution, Movable Type's Mena Trott, talks about the early days of blogging, when she realized that giving regular people the power to share our lives online is the key to building a friendlier, more connected world.


In [69]:
from transformers import GPT2Tokenizer, GPT2Model

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Function to compute cosine similarity
def calculate_chatgpt_similarity(df):
    similarities = []
    # Tokenize the texts
    for i in range(len(df)):
        inputs1 = tokenizer(df['Summary'][i], return_tensors='pt', padding=False, truncation=True)
        inputs2 = tokenizer(df['Original'][i], return_tensors='pt', padding=False, truncation=True)
        # Get embeddings for text1
        with torch.no_grad():
            outputs1 = model(**inputs1)
        embeddings1 = outputs1.last_hidden_state.mean(dim=1).to(device).squeeze().detach().cpu().numpy()

        # Get embeddings for text2
        with torch.no_grad():
            outputs2 = model(**inputs2)
        embeddings2 = outputs2.last_hidden_state.mean(dim=1).to(device).squeeze().detach().cpu().numpy()
        # Calculate cosine similarity
        embeddings1 = embeddings1.reshape(1, -1)
        embeddings2 = embeddings2.reshape(1, -1)
        similarity = cosine_similarity(embeddings1, embeddings2)[0]
        similarities.append(similarity)
    average_cosine_similarity = np.mean(similarities)
    return similarities, average_cosine_similarity


# Calculate cosine similarity
sim_chatgpt, avg_chatgpt  = calculate_chatgpt_similarity(gpt_summaries)

In [70]:
print(sim_chatgpt)
print(avg_chatgpt)

[array([0.99958193], dtype=float32), array([0.99933034], dtype=float32), array([0.9968271], dtype=float32), array([0.99844074], dtype=float32), array([0.9986124], dtype=float32), array([0.999394], dtype=float32), array([0.99881], dtype=float32), array([0.9993689], dtype=float32), array([0.9982764], dtype=float32), array([0.9987984], dtype=float32), array([0.9987745], dtype=float32), array([0.9991476], dtype=float32), array([0.9984833], dtype=float32), array([0.999375], dtype=float32), array([0.99939716], dtype=float32), array([0.9978632], dtype=float32), array([0.9987481], dtype=float32), array([0.9990869], dtype=float32), array([0.99495083], dtype=float32), array([0.9986136], dtype=float32), array([0.99872243], dtype=float32), array([0.9990887], dtype=float32), array([0.9985764], dtype=float32), array([0.99863], dtype=float32), array([0.9991771], dtype=float32), array([0.9963416], dtype=float32), array([0.9982637], dtype=float32), array([0.99554896], dtype=float32), array([0.9987023],

In [71]:
print(gpt_summaries.iloc[0,0])
print(gpt_summaries.iloc[0,1])

Mena Trott, a blogger, shares her journey of starting a blog in 2001 and the personal stories she shared, including a touching story of a fellow blogger who documented her battle with cancer. She discusses the power of blogging in connecting people and sharing personal experiences, emphasizing the importance of leaving a digital legacy through blogs. Mena reflects on the impact of blogs in shaping personal narratives and creating connections across the world, highlighting the significance of sharing stories through this medium. She encourages the audience to consider the power of blogs and to embrace the platform as a means of personal expression and connection.
The founding mother of the blog revolution, Movable Type's Mena Trott, talks about the early days of blogging, when she realized that giving regular people the power to share our lives online is the key to building a friendlier, more connected world.


In [72]:
gpt_summaries.to_csv("/content/drive/MyDrive/CSE6740/gpt_summaries.csv")